##Trend for hashtags

In [2]:
ACCESS_KEY = "xxx"
SECRET_KEY = "xxx".replace("/", "%2F")
AWS_BUCKET_NAME = "project4capstones3"
MOUNT_NAME = "twitter_19038028888888"

dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

import json
# input = sc.textFile("/mnt/twitter/2017/06/*/*/project*")
data = sqlContext.read.json("/mnt/twitter/2017/06/*/*/project*")

In [3]:
df = data.select('status_created_at', 'searched_names', 'status_sentScore')

In [4]:
from datetime import datetime, timedelta

def dt_parse(datetime_str):
  dt_tz = datetime_str.split()
  dt_str, tz_str = ' '.join(dt_tz[:4] + dt_tz[5:]), dt_tz[4]
  dt = datetime.strptime(dt_str, '%c')
  if tz_str[0] == '+':
    dt += timedelta(hours=int(tz_str[1:3]), minutes=int(tz_str[3:]))
  elif tz_str[0] == '-':
    dt -= timedelta(hours=int(tz_str[1:3]), minutes=int(tz_str[3:]))
  return dt #.strftime(format='%Y-%M-%d %H:%M:%S')

In [5]:
from pyspark.sql.types import TimestampType
dt_parse_udf =  udf(dt_parse, TimestampType())

df_final = df.withColumn('status_created_at', dt_parse_udf(df['status_created_at']))
# df_final.show()

In [6]:
from pyspark.sql.functions import hour, dayofyear

df_trend = df_final.select(dayofyear('status_created_at').alias('day'),hour('status_created_at').alias('hour'), "status_sentScore", 'searched_names').groupby('day', 'hour', 'searched_names').agg({"*":"count", "status_sentScore":"avg"}).sort('day', 'hour').withColumnRenamed("count(1)", "Tweet_Cnt").withColumnRenamed("avg(status_sentScore)", "Avg_Sent_Score")

In [7]:
from pyspark.sql.functions import concat, col, lit

# merging day and hour column and adding it as extra column
df_trend_final = df_trend.withColumn("day_hour", concat(col("day"),col("hour")))

In [8]:
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import json

In [9]:
df_trend_final.write.saveAsTable('trend_df', mode = 'overwrite')

In [13]:
# from pyspark.sql.functions import hour, dayofyear
# display(df_final.select(hour('status_created_at').alias('hour'), dayofyear('status_created_at').alias('day')).groupby('hour', 'day').count().sort('day', 'hour'))